In [1]:
!pip install emoji

     |████████████████████████████████| 133kB 30.0MB/s 


In [31]:
import pandas as pd
import numpy as np
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [32]:
!git clone https://github.com/amueller/word_cloud.git
%cd word_cloud
!pip install 

Cloning into 'word_cloud'...
remote: Enumerating objects: 6459, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 6459 (delta 8), reused 6 (delta 0), pack-reused 6441
Receiving objects: 100% (6459/6459), 116.93 MiB | 35.39 MiB/s, done.
Resolving deltas: 100% (3839/3839), done.
/content/word_cloud/word_cloud/word_cloud
ERROR: You must give at least one requirement to install (see "pip help install")


In [40]:
filepath = input("Please Enter the Whatsapp chat export path below \n*  eg. E:\IIIT-B\Projects\WhatsappGroupChatAnalyses\WhatsappGroupChatExport.txt \n")
df = pd.read_csv(filepath, sep = "delimiter",skip_blank_lines = True, header = None, engine= 'python')

Please Enter the Whatsapp chat export path below 
*  eg. E:\IIIT-B\Projects\WhatsappGroupChatAnalyses\WhatsappGroupChatExport.txt 
/content/drive/MyDrive/Colab Notebooks/WhatsApp Chat with Family.txt


# Data Preparation

In [43]:
# extract date values and return as list of string
def getdate(x):
    res = re.search("\d\d/\d\d/\d\d",x)
    if res != None:
        return res.group()
    else:
        return ""
#datepattern = re.compile("../../..")
df["Date"] = list(map(lambda x : getdate(x), df.iloc[:,0]))


In [44]:
## Merge multiline chat data
## Need to optimize this block
for i in range(0,len(df)):
    if df["Date"][i] == "":
        c=i-1
        for j in range(i,len(df)):
            if df["Date"][j] == "":
                df.iloc[c,0] = " ".join([df.iloc[c,0],df.iloc[j,0]])
                    
            else:
                i=j-1
                break
    else:
        df.iloc[i,0] = df.iloc[i,0]
        

In [45]:
## Remove rows where date is empty
df.drop(np.where(df.iloc[:,1]=="")[0],inplace =True)
## Reindex the dataframe
df.index = range(0,len(df))



In [46]:
##Remove date from original text data using substitute function of regular expression
df.iloc[:,0] = list(map(lambda x : re.sub("../../..","", x)[2:],df.iloc[:,0]))



In [47]:
## Extract Day Month and Year from Date 
df["Day"] = list(map(lambda d : d.split("/")[0], df.Date))
df["Months"] = list(map(lambda d : d.split("/")[1], df.Date))
df["Year"] = list(map(lambda d : d.split("/")[2], df.Date))

In [49]:
## extract time stamp from chat data and store in new column Time and Am
def gettime(x):
    res = re.search(".*\d:\d\d\s[a|p]m", x)
    if res != None:
        return res.group()
    else:
        return ""
Timestamp = list(map(lambda x : gettime(x),df.iloc[:,0])) 


In [50]:
df["Time"] = list(map(lambda t : t.split(" ")[0],Timestamp))
df["Hour"] = list(map(int,list(map(lambda t : t.split(":")[0],df["Time"]))))
df["Minute"] = list(map(int,list(map(lambda t : t.split(":")[1],df["Time"]))))
df["AmPm"] = list(map(lambda t : t.split(" ")[1],Timestamp))



In [51]:
## Remove Timestamps from chat
df.iloc[:,0] = list(map(lambda x : re.sub(".*\d:\d\d\s[a|p]m","", x)[2:],df.iloc[:,0]))


## get sender
def getsender(x):
    res = re.search(re.compile(".*?: "),x)
    if res !=None:
        return res.group()[1:-2]
    else:
        return ""
    
    
df["sender"] =list(map(getsender,df.iloc[:,0]))


In [52]:
## Drop rows of activity messages - member added/removed/left/group name change/icon change/others
df.drop((np.where(df["sender"]=="")[0]),inplace = True)
## Reindex the dataframe
df.index = range(0,len(df))


In [54]:
## extract final message from chat data
def getmessage(x):
    res = re.search(": .*",x)
    if res != None:
        return res.group()[2:]
    else:
        return None

df["Message"] = list(map(getmessage,df.iloc[:,0]))

## Drop column 0
df = df.drop(0,axis =1)

## Data Exploration

In [55]:
df.head()

,Date,Day,Months,Year,Time,Hour,Minute,AmPm,sender,Message
